In [20]:
import os
from glob import glob
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
import cv2
from pprint import pprint
import shutil

In [21]:
# chartqa = {
#     'notextBar': {
#         'answer': '13942',
#         'image_path': 'two_col_40186.png',
#         'options': ['3194', '13942', '14495', '28466'],
#         'question': "How many deaths resulted from cocaine use?"
#     },
#     'notextLine': {
#         'answer': '11.8%',
#         'image_path': 'two_col_4524.png',
#         'options': ['11.8%', '12.4%', '14.3%', '17%'],
#         'question': 'What is the poverty rate in California in the year 2019?'
#     },
#     'notextPie': {
#         'answer': '12%',
#         'image_path': 'two_col_61107.png',
#         'options': ['9%', '12%', '29%', '50%'],
#         'question': 'What is the percentage of "Very important"?'
#     },
#     'notextStackedBar': {
#         'answer': 'False',
#         'image_path': 'multi_col_60949.png',
#         'options': ['True', 'False'],
#         'question': 'The Y axis represents the date'
#     },
#     'textBar': {
#         'answer': 'Ireland',
#         'image_path': '05411753006467.png',
#         'options': ['Dominican Republic', 'Ireland', 'Guadeloupe'],
#         'question': 'Which country had the second largest value?'
#     },
#     'textLine': {
#         'answer': '1974',
#         'image_path': '19371621021871.png',
#         'options': ['1964', '1970', '1974', '1980'],
#         'question': 'In which year did public trust in government first drop below 50%?'
#     },
#     'textPie': {
#         'answer': 'True',
#         'image_path': '43.png',
#         'options': ['True', "False"],
#         'question': "Percentage of people who 'Do not use Latinx' is 20%"
#     },
#     'textStackedBar': {
#         'answer': '',
#         'image_path': '9280.png',
#         'options': ['17', '24', '40', '57'],
#         'question': "What's the percentage of people choosing 'Very' in Feb 2016?"
#     }
# }

In [22]:
# vlat_data = {
#     "TreeMap": {
#         "question": "The number of unique visitors for Amazon was more than that of Yahoo in 2010.",
#         "answer": "False",
#         "image_path": "TreeMap.png",
#         "options": ["True", "False"]
#     },
#     "Stacked100": {
#         "question": "The percentage of Gold medals of Australia is lower than Japan",
#         "answer": "True",
#         "image_path": "Stacked100.png",
#         "options": ["True", "False"]
#     },
#     "Histogram": {
#         "question": "More people traveled between 10 and 20km than between 60 and 70km.",
#         "answer": "True",
#         "image_path": "Histogram.png",
#         "options": ["True", "False"]
#     },
#     "Choropleth": {
#         "question": "In which state has the highest value?",
#         "answer": "Nevada (NV)",
#         "image_path": "Choropleth.png",
#         "options": ["Michigan (MI)", "Illinois (IL)", "Nevada (NV)", "New York (NY)"]
#     },
#     "PieChart": {
#         "question": "In which company is the global smartphone market share the smallest?",
#         "answer": "Vivo",
#         "image_path": "PieChart.png",
#         "options": ["Samsung", "Apple", "Vivo", "Xiaomi"]
#     },
#     "BubbleChart": {
#         "question": "The length of metro systems in this chart ranges from ______",
#         "answer": "160 - 460 km",
#         "image_path": "BubbleChart.png",
#         "options": ["160 - 460 km", "240 - 380 km", "240 - 560 km", "180 - 560 km"]
#     },
#     "StackedBar": {
#         "question": "The cost of vodka in Toronto is higher than that of Zurich.",
#         "answer": "True",
#         "image_path": "StackedBar.png",
#         "options": ["True", "False"]
#     },
#     "LineChart": {
#         "question": "Over the first three months in 2020, the price of a barrel of oil was ____________.",
#         "answer": "falling",
#         "image_path": "LineChart.png",
#         "options": ["rising", "falling", "staying the same"]
#     },
#     "BarChart": {
#         "question": "How many countries have a faster average internet speed than Taiwan?",
#         "answer": "4",
#         "image_path": "BarChart.png",
#         "options": ["3", "4", "5", "6"]
#     },
#     "AreaChart": {
#         "question": "Between July 2018 and July 2019, the average price of a pound of coffee beans ranged from _______ to ________",
#         "answer": "$0.71 - $0.87",
#         "image_path": "AreaChart.png",
#         "options": ["$0.61 - $0.75", "$0.75 - $0.85", "$0.71 - $0.87", "$0.76 - $0.85"]
#     },
#     "StackedArea": {
#         "question": "In the UK, the number of girls named \"Amelia\" was higher in 2014 than in 2013",
#         "answer": "False",
#         "image_path": "StackedArea.png",
#         "options": ["True", "False"]
#     },
#     "Scatterplot": {
#         "question": "What is the weight for the person who is 161cm tall?",
#         "answer": "44 kg",
#         "image_path": "Scatterplot.png",
#         "options": ["44 kg", "52 kg", "61 kg", "67 kg"]
#     }
# }

# additional_data = {
#    "TreeMap": {
#         "question": "Samsung is nested in the Financial category.",
#         "answer": "False",
#         "image_path": "VLAT-TreeMap.png",
#         "options": ["True", "False"]
#     },
#     "Stacked100": {
#         "question": "What is the education level of people in which the Democrats have the lowest approval rating?",
#         "answer": "College Graduate",
#         "image_path": "VLAT-Stacked100.png",
#         "options": ["High School Graduate or Less", "Some College Degree", "College Graduate", "Postgraduate study"]
#     },
#     "Histogram": {
#         "question": "Most people rated the taxi service between ______ and ______ stars",
#         "answer": "4.4 - 4.6",
#         "image_path": "VLAT-Histogram.png",
#         "options": ["4.2 - 4.4", "4.4 - 4.6", "4.6 - 4.8", "4.8 - 5.0"]
#     },
#     "Choropleth": {
#         "question": "In 2015, the unemployment rate for Wisconsin (WI) was higher than that of Washington (WA).",
#         "answer": "False",
#         "image_path": "VLAT-Choropleth.png",
#         "options": ["True", "False"]
#     },
#     "PieChart": {
#         "question": "The global smartphone market share of Apple is larger than that of Huawei.",
#         "answer": "True",
#         "image_path": "VLAT-PieChart.png",
#         "options": ["True", "False"]
#     },
#     "BubbleChart": {
#         "question": "Which city's metro system has the largest number of stations?",
#         "answer": "New York City",
#         "image_path": "VLAT-BubbleChart.png",
#         "options": ["Seoul", "Beijing", "New York City", "Shanghai"]
#     },
#     "StackedBar": {
#         "question": "What is the cost of peanuts in Las Vegas?",
#         "answer": "$12",
#         "image_path": "VLAT-StackedBar.png",
#         "options": ["$12", "$16.7", "$23.4", "$35.4"]
#     },
#     "LineChart": {
#         "question": "What was the price of a barrel of oil in February 2015?",
#         "answer": "$50.24",
#         "image_path": "VLAT-LineChart.png",
#         "options": ["$57.36", "$47.82", "$50.24", "$39.72"]
#     },
#     "BarChart": {
#         "question": "What is the average internet speed in Japan?",
#         "answer": "15 Mbps",
#         "image_path": "VLAT-BarChart.png",
#         "options": ["10 Mbps", "14 Mbps", "15 Mbps", "16 Mbps"]
#     },
#     "AreaChart": {
#         "question": "What was the average price of a pound of coffee beans in September 2013?",
#         "answer": "$5.1",
#         "image_path": "VLAT-AreaChart.png",
#         "options": ["$4.9", "$5.0", "$5.1", "$5.2"]
#     },
#     "StackedArea": {
#         "question": "Approximately, what is the ratio of the number of girls named 'Olivia' to those named 'Isla' in 2014 in the UK?",
#         "answer": "1 to 1",
#         "image_path": "VLAT-StackedArea.png",
#         "options": ["1 to 1", "1 to 2", "1 to 3", "1 to 4"]
#     },
#     "Scatterplot": {
#         "question": "There is a positive linear relationship between the height and the weight of the 85 males.",
#         "answer": "True",
#         "image_path": "VLAT-Scatterplot.png",
#         "options": ["True", "False"]
#     }
# }

In [54]:
# Transformed data structures (from list-of-singleton-dicts to dict-of-dicts):

original_vlat = {
    "treemap": {
        "question": "eBay is nested in the Software category.",
        "answer": "False",
        "image_path": "TreeMap.png",
        "options": ["True","False", "Cannot be inferred / inadequate information"]
    },
    "stackedBar100": {
        "question": "Which country has the lowest proportion of Gold medals?",
        "answer": "Great Britain",
        "image_path": "Stacked100.png",
        "options": ["Great Britain","USA","Japan", "None of Them"]
    },
    "histogram": {
        "question": "What distance have customers traveled in the taxi the most?",
        "answer": "30-40km",
        "image_path": "Histogram.png",
        "options": ["50-70km","30-40km","20-30km","50-60km"]
    },
    "choropleth": {
        "question": "In 2020, the unemployment rate for Washington (WA) was higher than that of Wisconsin (WI)?",
        "answer": "True",
        "image_path": "Choropleth.png",
        "options": ["True","False", "Cannot be inferred / inadequate information"]
    },
    "piechart": {
        "question": "What is the approximate global smartphone market share of Samsung?",
        "answer": "17.6%",
        "image_path": "PieChart.png",
        "options": ["17.6%","25.3%","10.9%","35.2%"]
    },
    "bubblechart": {
        "question": "Which city's metro system has the largest number of stations?",
        "answer": "Shanghai",
        "image_path": "BubbleChart.png",
        "options": ["Beijing","Shanghai","London", "None of Them"]
    },
    "stackedbar": {
        "question": "What is the cost of peanuts in Seoul?",
        "answer": "$6.1",
        "image_path": "StackedBar.png",
        "options": ["$5.2","$6.1","$7.5","$4.5"]
    },
    "linechart": {
        "question": "What was the price of a barrel of oil in February 2020?",
        "answer": "$50.54",
        "image_path": "LineChart.png",
        "options": ["$50.54","$47.02","$42.34","$43.48"]
    },
    "barchart": {
        "question": "What is the average internet speed in Japan?",
        "answer": "40.51 Mbps",
        "image_path": "BarChart.png",
        "options": ["42.30 Mbps","40.51 Mbps","35.25 Mbps","16.16 Mbps"]
    },
    "areachar": {
        "question": "What was the average price of a pound of coffee in October 2019?",
        "answer": "$0.71",
        "image_path": "AreaChart.png",
        "options": ["$0.71","$0.90","$0.80","$0.63"]
    },
    "stackedarea": {
        "question": "What was the ratio of girls named \"Isla\" to girls named \"Amelia\" in 2012 in the UK?",
        "answer": "1 to 2",
        "image_path": "StackedArea.png",
        "options": ["1 to 1","1 to 2","1 to 3","1 to 4"]
    },
    "scatterplot": {
        "question": "There is a negative relationship between the height and the weight of the 85 males.",
        "answer": "False",
        "image_path": "Scatterplot.png",
        "options": ["True","False", "Cannot be inferred / inadequate information"]
    }
}

calvi = {
    "calvi1": {
        "question": "What is the number of concert tickets sold on Aug 10 as a proportion of that on Aug 11?",
        "answer": "80%",
        "image_path": "calvi1.png",
        "options": ["20%","40%","60%","80%"]
    },
    "calvi2": {
        "question": "Predatory species Z is more prevalent in NC than in AZ.",
        "answer": "False",
        "image_path": "calvi2.png",
        "options": ["True","False","Cannot be inferred / inadequate information"]
    },
    "calvi3": {
        "question": "Which of the following is True about the annual number of new moview released in city X?",
        "answer": "There were more new movies released in 1992 than in 1994.",
        "image_path": "calvi3.png",
        "options": [
            "There were more new movies released in 1992 than in 1994.",
            "2000 had the most number of new movies released.",
            "There were more new movies released in 1996 than in 1995.",
            "None of the above"
        ]
    },
    "calvi4": {
        "question": "Which of the following is true about the prevalence of the plant species Y?",
        "answer": "It is less prevalent in region C than in D.",
        "image_path": "calvi4.png",
        "options": [
            "It is more prevalent in region D than in E.",
            "It is less prevalent in region B than in C.",
            "It is more prevalent in region C than in A.",
            "It is less prevalent in region C than in D."
        ]
    },
    "calvi5": {
        "question": "Which of the following is true about the three brands of shampoo in city Y from 2001 to 2005?",
        "answer": "The market share of brand C is higher in 2001 than in 2002.",
        "image_path": "calvi5.png",
        "options": [
            "The market share of brand C is on an increasing trend from 2001 to 2005.",
            "The market share of brand C is higher in 2001 than in 2002.",
            "Brand B has the largest market share in 2001 compared to its market share in the other years.",
            "None of the above."
        ]
    },
    "calvi6": {
        "question": "Approximately, what is the numver of votes for candidate A as a proportion of those for candidate B?",
        "answer": "80%",
        "image_path": "calvi6.png",
        "options": ["50%","60%","70%","80%"]
    },
    "calvi7": {
        "question": "Was the average amount of precipitation over 15 years in region D higher than that in region C?",
        "answer": "Cannot be inferred / inadequate information",
        "image_path": "calvi7.png",
        "options": ["Yes","No","Cannot be inferred / inadequate information"]
    },
    "calvi8": {
        "question": "Assuming today is Feb 20, 2022, which of the following is true about the vegetables planted by the three farms?",
        "answer": "None of the above.",
        "image_path": "calvi8.png",
        "options": [
            "All three farms planted more vegetables in 2022 than they did in the previous years.",
            "All three farms planted fewer vegetables in 2022 than they did in the previous years.",
            "All three farms planted about the same amount of vegetables in 2022 than they did in the previous years.",
            "None of the above."
        ]
    },
    "calvi9": {
        "question": "Assuming today is May 01, 2022, which of the following statements is true?",
        "answer": "None of the above.",
        "image_path": "calvi9.png",
        "options": [
            "Company Z had more employees in 2018 than in 2020.",
            "The number of new employees per year fell sharply in 2022.",
            "The total number of employees in company Z decreased in 2022.",
            "None of the above."
        ]
    },
    "calvi10": {
        "question": "Does any member of species C weigh more than 5 lbs?",
        "answer": "Cannot be inferred / inadequate information",
        "image_path": "calvi10.png",
        "options": ["Yes","No","Cannot be inferred / inadequate information"]
    },
    "calvi11": {
        "question": "Which region had the highest number of confirmed cases per 1,000 people?",
        "answer": "Cannot be inferred / inadequate information",
        "image_path": "calvi11.png",
        "options": ["A","B","C","Cannot be inferred / inadequate information"]
    },
    "calvi12": {
        "question": "what is the trend of sales in gitt shop A from Jan to Dec?",
        "answer": "Cannot be inferred / inadequate information",
        "image_path": "calvi12.png",
        "options": ["Generally increasing","Generally decreasing","Generally constant","Cannot be inferred / inadequate information"]
    },
    "calvi13": {
        "question": "More than half of the people in country Z prefer reading at night.",
        "answer": "False",
        "image_path": "calvi13.png",
        "options": ["True","False","Cannot be inferred / inadequate information"]
    },
    "calvi14": {
        "question": "the residents of town Y area voting on a favorite hiking trail, and half of the votes have been counted. The poll closes on Sep 01, 2022. Which of the following is true?",
        "answer": "Too early to say.",
        "image_path": "calvi14.png",
        "options": [
            "Hiking trail A will be voted as the favorite trail.",
            "Hiking trail B will be voted as the favorite trail.",
            "Hiking trail C will be voted as the favorite trail.",
            "Too early to say."
        ]
    },
    "calvi15": {
        "question": "In town X, what is the approximate average number of customers at mall A from Jan to Mar?",
        "answer": "400",
        "image_path": "calvi15.png",
        "options": ["200","300","400","500"]
    }
}

In [57]:

def create_base_components():
    return {
        "bubble": {
            "type": "react-component",
            "path": "literacy/assets/bubble.jsx",
            "instructionLocation": "sidebar",
            "nextButtonLocation": "sidebar",
            "nextButtonDisableTime": 60000,
            "nextButtonEnableTime": 5000
        },
    }

In [76]:

def create_default_components():
    return {
        "instructions": {
            "type": "markdown",
            "path": "literacy/assets/instructions.md",
            "response": []
        },
        "consent": {
            "type": "markdown",
            "path": "shared/consent.md",
            "nextButtonText": "I agree",
            "response": []
        },
        # "vlat_intro": {
        #     "type": "markdown",
        #     "path": "literacy/assets/vlat_intro.md",
        #     "response": []
        # },
        "demographics": {
            "type": "markdown",
            "path": "shared/blank.md",
            "response": [
                {
                    "id": "gender",
                    "prompt": "What is your **gender**?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "withOther": True,
                    "options": [
                        "Woman",
                        "Man",
                        "Prefer not to say"
                    ],
                    "withDivider": True

                },
                {
                    "id": "age",
                    "prompt": "What is your **age**?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "Under 18 years",
                        "18-24 years",
                        "25-34 years",
                        "35-44 years",
                        "45-54 years",
                        "55-64 years",
                        "65 years or older",
                        "Prefer not to say"
                    ],
                    "withDivider": True
                },
                {
                    "id": "education",
                    "prompt": "What is the **highest degree or level of education** you have completed?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "withOther": True,
                    "options": [
                        "Less than high school",
                        "High school diploma or equivalent",
                        "Bachelor's degree or equivalent",
                        "Master's degree or equivalent",
                        "Doctoral degree or equivalent"
                    ],
                    "withDivider": True
                },
                {
                    "id": "experience",
                    "prompt": "Please briefly describe your experience with data visualizations.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "I create data visualizations frequently, either for work or as a hobby.",
                        "I create data visualizations sometimes, either for work or as a hobby.",
                        "I read data visualizations frequently, such as in news articles.",
                        "I read data visualizations sometimes, such as in news articles.",
                        "I rarely interact with data visualizations in my life."
                    ],
                    "withDivider": True
                }
            ]
        },
        "self_assessment": {
            "type": "markdown",
            "path": "shared/blank.md",
            "response": [
                {
                    "id": "SGL-item-1",
                    "prompt": "How good are you at working with bar charts? (1 = not at all good; 6 = extremely good)",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "1 (not at all good)",
                        "2",
                        "3",
                        "4",
                        "5",
                        "6 (extremely good)"
                    ],
                    "withDivider": True
                },
                {
                    "id": "SGL-item-2",
                    "prompt": "How good are you at working with line plots? (1 = not at all good; 6 = extremely good)",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "1 (not at all good)",
                        "2",
                        "3",
                        "4",
                        "5",
                        "6 (extremely good)"
                    ],
                    "withDivider": True
                },
                {
                    "id": "SGL-item-3",
                    "prompt": "How good are you at working with pies? (1 = not at all good; 6 = extremely good)",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "1 (not at all good)",
                        "2",
                        "3",
                        "4",
                        "5",
                        "6 (extremely good)"
                    ],
                    "withDivider": True
                },
                {
                    "id": "SGL-item-4",
                    "prompt": "How good are you at inferring the size of a bar in a bar chart? (1 = not at all good; 6 = extremely good)",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "1 (not at all good)",
                        "2",
                        "3",
                        "4",
                        "5",
                        "6 (extremely good)"
                    ],
                    "withDivider": True
                },
                {
                    "id": "SGL-item-5",
                    "prompt": "How good are you at determining the difference between 2 bars in a bar chart? (1 = not at all good; 6 = extremely good)",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "1 (not at all good)",
                        "2",
                        "3",
                        "4",
                        "5",
                        "6 (extremely good)"
                    ],
                    "withDivider": True
                },
                {
                    "id": "SGL-item-6",
                    "prompt": "How good are you at projecting a future trend from a line chart? (1 = not at all good; 6 = extremely good)",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "1 (not at all good)",
                        "2",
                        "3",
                        "4",
                        "5",
                        "6 (extremely good)"
                    ],
                    "withDivider": True
                },
                {
                    "id": "SGL-item-7",
                    "prompt": "Are graphs easier to understand than numbers? (1 = not at all; 6 = much easier)",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "1 (not at all)",
                        "2",
                        "3",
                        "4",
                        "5",
                        "6 (much easier)"
                    ],
                    "withDivider": True
                },
                {
                    "id": "SGL-item-8",
                    "prompt": "How often do you find graphical information to be useful? (1 = never; 6 = very often)",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "1 (never)",
                        "2",
                        "3",
                        "4",
                        "5",
                        "6 (very often)"
                    ],
                    "withDivider": True
                },
                {
                    "id": "SGL-item-9",
                    "prompt": "To what extent do you believe in the saying 'a picture is worth one thousand words'? (1 = not at all; 6 = extremely)",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "1 (not at all)",
                        "2",
                        "3",
                        "4",
                        "5",
                        "6 (extremely)"
                    ],
                    "withDivider": True
                },
                {
                    "id": "SGL-item-10",
                    "prompt": "When reading books or newspapers, how helpful do you find graphs that are part of a story? (1 = not at all; 6 = extremely)",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "1 (not at all)",
                        "2",
                        "3",
                        "4",
                        "5",
                        "6 (extremely)"
                    ],
                    "withDivider": True
                }
            ]

        }
    }


def create_bubble_components(data, tag='basic'):
    tasks = dict()
    for chart_type, value in data.items():
        question = value["question"]
        image_path = value["image_path"]
        options = value["options"]
        answer = value["answer"]

        tasks[f"{chart_type}-{tag}"] = {
            "baseComponent": "bubble",
            "instruction": f" **{question}**",
            "parameters": {
                "image": f"../literacy/assets/images/{image_path}",
                "question": question,
                "radius": 32,
            },
            "response": [
                {
                    "id": f"{chart_type}-{tag}",
                    "prompt": "Your answer",
                    "required": True,
                    "location": "sidebar",
                    "type": "radio",
                    "options": options + ["Skip"]
                },
            ],
            "correctAnswer": [
                {
                    "id": f"{chart_type}-{tag}",
                    "answer": answer
                }
            ]
        }

    return tasks

In [77]:
vlat_components = create_bubble_components(original_vlat, 'VLAT')
calvi_components = create_bubble_components(calvi, 'CALVI')

In [78]:
def sequence_generator():
    sequence = {
        "order": "fixed",
        "components": [
            "consent",
            "self_assessment",
            "instructions",
            {
                "id": "batch",
                "order": "random",
                "components": list(vlat_components.keys()) + list(calvi_components.keys())
            },
            "demographics",
            
        ]
    }
    return sequence

In [79]:
prolificRedirection = "https://app.prolific.com/submissions/complete?cc=CC8EMRUG"

In [80]:
default_components = create_default_components()
baseComponents = create_base_components()
components = default_components | vlat_components | calvi_components
sequence = sequence_generator()
print(f"Total number of components: {len(components)}")

Total number of components: 31


In [81]:
with open("config.json", "r") as f:
    config = json.load(f)
config['uiConfig']['studyEndMsg'] = f"**Thank you for completing the study. You may click this link and return to Prolific**: [{prolificRedirection}]({prolificRedirection})"
config['components'] = components
config['sequence'] = sequence
config['baseComponents'] = baseComponents
with open("config.json", "w") as f:
    json.dump(config, f, indent=4)